In [2]:
import csv
from datetime import datetime, timezone
from collections import deque
import numpy as np
from sklearn.linear_model import LogisticRegression

In [3]:
def load_binance_csv(path):
    candles = []
    with open(path, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            candles.append({
                "timestamp": int(row[0]) // 1000,
                "datetime": datetime.fromtimestamp(int(row[0]) // 1_000_000, timezone.utc),
                "open": float(row[1]),
                "high": float(row[2]),
                "low": float(row[3]),
                "close": float(row[4]),
                "volume": float(row[5]),
            })
    return candles

path = "../data/BTCUSDT-1m-2025-05.csv"  # update to your filename
candles = load_binance_csv(path)

In [11]:
from analytics.MOMENTUM import (
    sma_crossover_strategy,
    ema_momentum_strategy,
    roc_momentum_strategy,
    macd_strategy,
    rsi_strategy,
    bollinger_bands_strategy,
    momentum_with_volume_strategy,
    markov_price_strategy,
)

In [12]:
def generate_training_data(candles, strategy_funcs, window_size=300):
    features = []
    labels = []
    window = deque(maxlen=window_size)

    for i in range(len(candles) - 1):
        window.append(candles[i])
        if len(window) < window_size:
            continue

        signals = [f(window) for f in strategy_funcs]
        feat_row = []
        for s in signals:
            # Use confidence if buy or sell, else 0
            feat_row.append(s["confidence"] if s["signal"] == "buy" else 0)
            feat_row.append(s["confidence"] if s["signal"] == "sell" else 0)

        next_close = candles[i + 1]["close"]
        current_close = candles[i]["close"]
        label = 1 if next_close > current_close else 0

        features.append(feat_row)
        labels.append(label)

    return np.array(features), np.array(labels)


def train_aggregator(features, labels):
    model = LogisticRegression()
    model.fit(features, labels)
    return model


def extract_strategy_weights(model, num_strategies):
    weights = model.coef_[0]
    strategy_weights = []
    for i in range(num_strategies):
        buy_weight = weights[2 * i]
        sell_weight = weights[2 * i + 1]
        strategy_weights.append({"buy": buy_weight, "sell": sell_weight})
    return strategy_weights

In [14]:
strategies = [
        sma_crossover_strategy,
        ema_momentum_strategy,
        roc_momentum_strategy,
        macd_strategy,
        rsi_strategy,
        bollinger_bands_strategy,
        momentum_with_volume_strategy,
        markov_price_strategy,
    ]

print("Generating training data...")
features, labels = generate_training_data(candles, strategies)

print("Training logistic regression aggregator...")
model = train_aggregator(features, labels)

print("Extracting strategy weights...")
weights = extract_strategy_weights(model, len(strategies))

for i, w in enumerate(weights):
    print(f"Strategy {i+1}: Buy Weight = {w['buy']:.4f}, Sell Weight = {w['sell']:.4f}")

Generating training data...
Training logistic regression aggregator...
Extracting strategy weights...
Strategy 1: Buy Weight = 0.0324, Sell Weight = 0.0380
Strategy 2: Buy Weight = -0.0530, Sell Weight = 0.0340
Strategy 3: Buy Weight = 0.0018, Sell Weight = 0.0059
Strategy 4: Buy Weight = -0.0043, Sell Weight = -0.0068
Strategy 5: Buy Weight = -0.0019, Sell Weight = -0.0763
Strategy 6: Buy Weight = -0.0001, Sell Weight = 0.0003
Strategy 7: Buy Weight = 0.0206, Sell Weight = 0.1004
Strategy 8: Buy Weight = 0.0075, Sell Weight = -0.1202


In [15]:
weights

[{'buy': np.float64(0.03244280654346815),
  'sell': np.float64(0.03804849089987302)},
 {'buy': np.float64(-0.05297275052732096),
  'sell': np.float64(0.03397235873034015)},
 {'buy': np.float64(0.001761384766475957),
  'sell': np.float64(0.005894839787707059)},
 {'buy': np.float64(-0.004270414821777813),
  'sell': np.float64(-0.0068446179029911895)},
 {'buy': np.float64(-0.001857576623100386),
  'sell': np.float64(-0.07633625060985212)},
 {'buy': np.float64(-8.465059911680986e-05),
  'sell': np.float64(0.00034531225160149215)},
 {'buy': np.float64(0.020615215447378748),
  'sell': np.float64(0.10036010981203138)},
 {'buy': np.float64(0.0075037459512008724),
  'sell': np.float64(-0.12022576934241866)}]